In [1]:
from google.cloud import bigquery 
client = bigquery.Client() ## Creating a bigquery client object!!


Using Kaggle's public dataset BigQuery integration.


In [2]:
# Reference to openaq dataset
dataset_ref = client.dataset("openaq", project="bigquery-public-data")

# API Request
dataset = client.get_dataset(dataset_ref)

## List all the tables in the dataset!!
tables = list(client.list_tables(dataset))

## display them
for table in tables:
    print(table.table_id)


global_air_quality


In [3]:
## Creating a reference to table 
table_ref = dataset_ref.table("global_air_quality")
## Api REQUEST
table = client.get_table(table_ref)

## Lets preview them
client.list_rows(table, max_results=5).to_dataframe()

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours,location_geom
0,"Borówiec, ul. Drapałka",Borówiec,PL,bc,0.85217,2022-04-28 07:00:00+00:00,µg/m³,GIOS,1.0,52.276794,17.074114,POINT(52.276794 1)
1,"Kraków, ul. Bulwarowa",Kraków,PL,bc,0.91284,2022-04-27 23:00:00+00:00,µg/m³,GIOS,1.0,50.069308,20.053492,POINT(50.069308 1)
2,"Płock, ul. Reja",Płock,PL,bc,1.41000,2022-03-30 04:00:00+00:00,µg/m³,GIOS,1.0,52.550938,19.709791,POINT(52.550938 1)
3,"Elbląg, ul. Bażyńskiego",Elbląg,PL,bc,0.33607,2022-05-03 13:00:00+00:00,µg/m³,GIOS,1.0,54.167847,19.410942,POINT(54.167847 1)
4,"Piastów, ul. Pułaskiego",Piastów,PL,bc,0.51000,2022-05-11 05:00:00+00:00,µg/m³,GIOS,1.0,52.191728,20.837489,POINT(52.191728 1)


In [4]:
## Query to select all the items from the city columns where the 
## "country" columns is "US"

query = """
SELECT city
FROM `bigquery-public-data.openaq.global_air_quality`
WHERE country = 'US'
"""

In [5]:
query_job = client.query(query)


In [6]:
us_cities = query_job.to_dataframe()


In [7]:
us_cities.city.value_counts().head()

city
Phoenix-Mesa-Scottsdale                     39414
Los Angeles-Long Beach-Santa Ana            27479
Riverside-San Bernardino-Ontario            26887
New York-Northern New Jersey-Long Island    25417
San Francisco-Oakland-Fremont               22710
Name: count, dtype: int64

In [8]:
query = """
        SELECT city, country
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        
"""

In [9]:
query = """
    
    SELECT score, title
    FROM `bigquery-public-data.hacker_news.full`
    WHERE type="job"

"""

In [10]:
#create a QueryJobconfig object to estimate size of query without running it

dry_run_config = bigquery.QueryJobConfig(dry_run=True)

## API REQUEST

dry_run_query_job = client.query(query, job_config=dry_run_config)

print("This query will process {} bytes".format(dry_run_query_job.total_bytes_processed))


This query will process 490833096 bytes


In [11]:
# Only run the query if it's less than 1 GB
ONE_GB = 1000*1000*1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_GB)

# Set up the query (will only run if it's less than 1 GB)
safe_query_job = client.query(query, job_config=safe_config)

# API request - try to run the query, and return a pandas DataFrame
job_post_scores = safe_query_job.to_dataframe()

# Print average score for job posts
job_post_scores.score.mean()

1.700647369595952